<a href="https://colab.research.google.com/github/Base-R-Best-R/Auction/blob/main/Code/Models/Colab/RF_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# call R in Python
%reload_ext rpy2.ipython

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# R
%%R

# source
source("drive/MyDrive/Master_Thesis/Auxilliary/Auxilliary.R")

# load data 
dat_bids_id <- readRDS("drive/MyDrive/Master_Thesis/Input_Data/Bids_id_df_split.RDS")

# exclude variables that are not supposed to be in the model
lapply(dat_bids_id, \(x){

  # remove
  # x$Contract_ID <- NULL
  x$Vendor_Name <- NULL

  # return
  return(x)
  
}) -> dat_bids_id 

# trainings data set
dat_train <- dat_bids_id[["Train"]]

# dependent var to factor
dat_train[, "Win"] <- as.factor(dat_train[, "Win"])

# packages
library(caret)
library(ranger)

R[write to console]: Loading required package: ggplot2

R[write to console]: Loading required package: lattice



In [ ]:
# R
%%R

# check factor levels to assess max value for mtry
rapply(dat_train, classes = "factor", f = levels) |> length()

# control
ctrl <- caret::trainControl(method = "repeatedcv", number = 5, repeats = 2)

# tuning grid
tgrid <- expand.grid("mtry" = seq(25, 700, 25),
                     "splitrule" = "gini",
                     "min.node.size" = 1:3)

# cross validation
rf <- caret::train(Win ~ ., data = dat_train, method = "ranger",
                   metric = "Kappa", tuneGrid = tgrid, trControl = ctrl)

# save
saveRDS(rf, "drive/MyDrive/Master_Thesis/Models_MT/cvfit_bid_id.RDS")


Growing trees.. Progress: 94%. Estimated remaining time: 2 seconds.
Growing trees.. Progress: 96%. Estimated remaining time: 1 seconds.
Growing trees.. Progress: 88%. Estimated remaining time: 4 seconds.
Growing trees.. Progress: 81%. Estimated remaining time: 7 seconds.
Growing trees.. Progress: 77%. Estimated remaining time: 9 seconds.
Growing trees.. Progress: 76%. Estimated remaining time: 9 seconds.
Growing trees.. Progress: 75%. Estimated remaining time: 10 seconds.
Growing trees.. Progress: 68%. Estimated remaining time: 14 seconds.
Growing trees.. Progress: 68%. Estimated remaining time: 14 seconds.
Growing trees.. Progress: 67%. Estimated remaining time: 15 seconds.
Growing trees.. Progress: 65%. Estimated remaining time: 16 seconds.
Growing trees.. Progress: 66%. Estimated remaining time: 16 seconds.
Growing trees.. Progress: 56%. Estimated remaining time: 24 seconds.
Growing trees.. Progress: 93%. Estimated remaining time: 2 seconds.
Growing trees.. Progress: 87%. Estimated 

In [ ]:
# unmount drive 
drive.flush_and_unmount()